<a href="https://colab.research.google.com/github/SergeyB81/Portfolio-DS/blob/main/Car%20price/useful_stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
! gdown  --id 1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ym8EaWePP17RvrweKhWzCJmITXLgqvCZ
To: /content/churn-modeling.zip
100% 268k/268k [00:00<00:00, 13.4MB/s]


In [15]:
import pandas as pd

In [16]:
df = pd.read_csv('/content/churn-modeling.zip')

In [17]:
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# **Обучающая, валидационная и тестовая выборка**

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train, test = train_test_split(df,train_size=0.6,random_state=42,stratify=df['Exited'])

In [21]:
val, test = train_test_split(test,train_size=0.5,random_state=42,stratify=test['Exited'])

# **Обучение модели**

In [22]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.7 MB/s eta 0:00:00


In [23]:
from catboost import CatBoostClassifier

In [29]:
df.dtypes

,0
RowNumber,int64
CustomerId,int64
Surname,object
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64


In [30]:
df.select_dtypes(include='object')

,Surname,Geography,Gender
0,Hargrave,France,Female
1,Hill,Spain,Female
2,Onio,France,Female
3,Boni,France,Female
4,Mitchell,Spain,Female
...,...,...,...
9995,Obijiaku,France,Male
9996,Johnstone,France,Male
9997,Liu,France,Female
9998,Sabbatini,Germany,Male


In [31]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [32]:
X = ['CustomerId', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

cat_features = ['Geography','Gender']

y = ['Exited']

In [53]:
from catboost import Pool

train_data = Pool(data=train[X],
                  label=train[y],
                  cat_features=cat_features
                  )

valid_data = Pool(data=val[X],
                  label=val[y],
                  cat_features=cat_features
                  )

test_data = Pool(data=test[X],
                  label=test[y],
                  cat_features=cat_features
                  )



In [38]:
params = {'verbose':100,
          'eval_metric':'AUC',
          'loss_function':'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [39]:
model = CatBoostClassifier(**params)

In [40]:
model.fit(train_data,eval_set=valid_data)

0:	test: 0.8338545	best: 0.8338545 (0)	total: 57.9ms	remaining: 57.8s
100:	test: 0.8725090	best: 0.8725090 (100)	total: 1.38s	remaining: 12.3s
200:	test: 0.8784656	best: 0.8784656 (200)	total: 2.13s	remaining: 8.45s
300:	test: 0.8812676	best: 0.8812676 (300)	total: 2.87s	remaining: 6.67s
400:	test: 0.8823868	best: 0.8824376 (390)	total: 3.66s	remaining: 5.47s
500:	test: 0.8834491	best: 0.8834522 (498)	total: 4.43s	remaining: 4.41s
600:	test: 0.8834737	best: 0.8836554 (564)	total: 5.19s	remaining: 3.44s
700:	test: 0.8837847	best: 0.8838694 (685)	total: 6.23s	remaining: 2.66s
800:	test: 0.8834383	best: 0.8838694 (685)	total: 7.95s	remaining: 1.98s
900:	test: 0.8829318	best: 0.8838694 (685)	total: 8.87s	remaining: 975ms
999:	test: 0.8822744	best: 0.8838694 (685)	total: 9.64s	remaining: 0us

bestTest = 0.8838694083
bestIteration = 685

Shrink model to first 686 iterations.


In [41]:
n_iters = model.best_iteration_ + 1

In [42]:
n_iters

686

In [43]:
params = {'iterations':n_iters,
          'verbose':100,
          'eval_metric':'AUC',
          'loss_function':'Logloss',
          'random_seed':42,
          'learning_rate':0.01}

In [44]:
model = CatBoostClassifier(**params)

In [45]:
train_full = pd.concat([train,val])

In [46]:
train_full_data = Pool(data=train_full[X],
                  label=train_full[y],
                  cat_features=cat_features
                  )

In [47]:
model.fit(train_full_data)

0:	total: 30.3ms	remaining: 20.7s
100:	total: 1.42s	remaining: 8.2s
200:	total: 2.28s	remaining: 5.5s
300:	total: 3.15s	remaining: 4.02s
400:	total: 4.98s	remaining: 3.54s
500:	total: 6.4s	remaining: 2.36s
600:	total: 7.28s	remaining: 1.03s
685:	total: 8.03s	remaining: 0us


In [54]:
test['y_score_no_cross_val'] = model.predict_proba(test_data)[:,1]

In [55]:
from sklearn.metrics import roc_auc_score

In [57]:
roc_auc_score(test['Exited'],test['y_score_no_cross_val'])

np.float64(0.8735839074822127)